In [ ]:
import datapane as dp
dp.enable_logging()
dp.client.log.setLevel("INFO")
dp.__rev__

In [ ]:
import datetime as dt
import humanize
def since_now(d: dt.datetime | None) -> str:
    if d is None:
        return "Never"
    return humanize.naturaldelta(dt.datetime.utcnow() - d)

In [ ]:
from collections import Counter

global_said_hello = Counter()

message = """
# Hello, {name}!
Your session started {session_age} ago

## Your past visits
{your_past_visits}

## Others around
{others_past_visits}
"""

def to_past_vists(c: Counter[str]) -> str:
    return "\n".join(f"- {k} ({v})" for k, v in c.most_common())

def say_hello(params, session: dict):
    name = params["foo"]
    session_age = session.setdefault("age", dt.datetime.utcnow())
    said_hello = session.setdefault("said_hello", Counter())
    global_said_hello.update([name])
    said_hello.update([name])
    return dp.View(
        dp.Text(message.format(
           name=name,
           your_past_visits=to_past_vists(session["said_hello"]),
           others_past_visits=to_past_vists(global_said_hello),
           session_age=since_now(session_age)
        ))
    )


In [ ]:
import datetime as dt
start_time = dt.datetime.utcnow()
last_viewed = None


def view():
    global last_viewed
    v = dp.View(
        dp.Text(f"## Booted: {since_now(start_time)} ago"),
        dp.Text(f"## Last Viewed: {since_now(last_viewed)} ago"),
        dp.Group(
            dp.Function(
                say_hello,
                target="result",
                controls=[
                    dp.TextBox(name="foo", label="Name"),
                ],
                label="Say Hello!",
            ),
            dp.Empty(name="result"),
            columns=2,
        )
    )
    last_viewed = dt.datetime.utcnow()
    return v

In [ ]:
dp.serve(view)